In [1]:
import numpy as np
import os
import sys

sys.path.append("..")
from dataloader import load_raw, create_datasets
from measures import quantile_loss, quantile_loss_sample, compute_intermittent_indicators

In [2]:
dataset_name = "M5"
data_raw, data_info = load_raw(dataset_name, datasets_folder_path=os.path.join("..","..","data"))
adi, cv2 = compute_intermittent_indicators(data_raw, data_info['h'])
datasets = create_datasets(data_raw, data_info)

lag = data_info['h']*data_info['w']
dlen = {
    'carparts':2489,
    'OnlineRetail':2023,
    'Auto':1227,
    'RAF':5000,
    'M5':29003
}
filter2 = np.load("mask_onlineretail.npy") if dataset_name == "OnlineRetail" else np.ones(dlen[dataset_name], dtype=bool)


quantiles = np.array([0.5, 0.8, 0.9, 0.95, 0.99])

filter, filter_label = adi >= 1.32, "intermittent_and_lumpy"

tmp = np.empty(shape=(len(datasets['test']), len(datasets['valid'][0]['target']), quantiles.size))
for i in range(len(datasets['test'])):
    tmp[i, :] = np.round(np.quantile(datasets['valid'][i]['target'], q=quantiles))
res_base_scale_tmp = []
for i in range(len(datasets['test'])):
    res_base_scale_tmp.append(quantile_loss(
        np.array(datasets['valid'][i]['target']).reshape(1,-1), 
        tmp[i].reshape(1,tmp[i].shape[0],tmp[i].shape[1]), 
        quantiles, avg=False))
res_base_scale = {}
for q in ['QL50','QL80','QL90','QL95','QL99']:
    res_base_scale[q] = np.mean(np.vstack([res_base_scale_tmp[i][q] for i in range(len(datasets['test']))]), axis=1)

scale = True
fscale = lambda x, q: x / res_base_scale[q][filter, np.newaxis][filter2] if scale else x

In [3]:
import os

home_dir = os.path.expanduser("~")

res_twgp = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "TweedieGP", dataset_name, "qforecasts.npy"))[filter][filter2]
if dataset_name == "M5":
    actuals = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{1}__{dataset_name}", "actuals.npy"))[filter][filter2]
    # baselines
    res_empq = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "EmpQ__M5__2024-08-09-09-50-34-215623", "qforecasts.npy"))[filter][filter2]
    res_iets = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "iETS__M5__2024-09-10-10-31-46-619048", "qforecasts.npy"))[filter][filter2]
    # globals
    res_glm = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "discrete-regression", f"NBs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    res_fnn = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    # globals boosted
    res_dnnNB = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "transformer__M5__negbin__mean-demand__2024-06-21-13-47-45-072202", "forecasts.npy"))[filter][filter2]
    res_dnnTW = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "transformer__M5__tweedie__mean-demand__2024-06-21-13-47-45-074833", "forecasts.npy"))[filter][filter2]
elif dataset_name == "OnlineRetail":
    actuals = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{1}__{dataset_name}", "actuals.npy"))[filter][filter2]
    # baselines
    res_empq = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "EmpQ__OnlineRetail__2024-08-09-09-54-07-013545", "qforecasts.npy"))[filter][filter2]
    res_iets = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "iETS__OnlineRetail__2024-09-10-10-25-50-415303", "qforecasts.npy"))[filter][filter2]
    # globals
    res_glm = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "discrete-regression", f"NBs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    res_fnn = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    # globals boosted
    res_dnnNB = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__OnlineRetail__negbin__mean-demand__2024-06-22-18-04-03-762960", "forecasts.npy"))[filter][filter2]
    res_dnnTW = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__OnlineRetail__tweedie__mean-demand__2024-06-22-18-42-17-982152", "forecasts.npy"))[filter][filter2]
elif dataset_name == "Auto":
    actuals = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{1}__{dataset_name}", "actuals.npy"))[filter][filter2]
    # baselines
    res_empq = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "EmpQ__Auto__2024-08-09-09-52-07-174894", "qforecasts.npy"))[filter][filter2]
    res_iets = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "iETS__Auto__2024-09-10-10-21-08-184429", "qforecasts.npy"))[filter][filter2]
    # globals
    res_glm = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "discrete-regression", f"NBs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    res_fnn = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    # globals boosted
    res_dnnNB = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__Auto__negbin__mean-demand__2024-06-22-11-04-18-074667", "forecasts.npy"))[filter][filter2]
    res_dnnTW = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__Auto__tweedie__mean-demand__2024-06-22-10-57-08-591613", "forecasts.npy"))[filter][filter2]
elif dataset_name == "RAF":
    actuals = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{1}__{dataset_name}", "actuals.npy"))[filter][filter2]
    # baselines
    res_empq = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "EmpQ__RAF__2024-08-09-09-54-23-309428", "qforecasts.npy"))[filter][filter2]
    res_iets = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "iETS__RAF__2024-09-10-10-22-33-114237", "qforecasts.npy"))[filter][filter2]
    # globals
    res_glm = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "discrete-regression", f"NBs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    res_fnn = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    # globals boosted
    res_dnnNB = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__RAF__negbin__mean-demand__2024-06-23-14-21-47-890960", "forecasts.npy"))[filter][filter2]
    res_dnnTW = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__RAF__tweedie__mean-demand__2024-06-23-14-31-11-241740", "forecasts.npy"))[filter][filter2]
elif dataset_name == "carparts":
    actuals = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{1}__{dataset_name}", "actuals.npy"))[filter][filter2]
    # baselines
    res_empq = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "EmpQ__carparts__2024-08-09-09-54-38-830033", "qforecasts.npy"))[filter][filter2]
    res_iets = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models_baselines", "iETS__carparts__2024-09-10-10-19-39-961801", "qforecasts.npy"))[filter][filter2]
    # globals
    res_glm = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "discrete-regression", f"NBs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    res_fnn = np.load(os.path.join(home_dir, "Codebase", "iTS", "src", "dnn-regression", f"DNNs_newl{lag}__{dataset_name}", "qforecasts.npy"))[filter][filter2]
    # globals boosted
    res_dnnNB = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__carparts__negbin__mean-demand__2024-06-23-10-35-39-750798", "forecasts.npy"))[filter][filter2]
    res_dnnTW = np.load(os.path.join(home_dir, "switchdrive", "iTS", "trained_models", "deepAR__carparts__tweedie__mean-demand__2024-06-23-09-25-09-068940", "forecasts.npy"))[filter][filter2]


In [4]:
# res_twgp = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/TweedieGP/{dataset_name}/qforecasts.npy")[filter][filter2]
# if dataset_name == "M5":
#     actuals = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{1}__{dataset_name}/actuals.npy")[filter][filter2]
#     # baselines
#     res_empq = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/EmpQ__M5__2024-08-09-09-50-34-215623/qforecasts.npy")[filter][filter2]
#     res_iets = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/iETS__M5__2024-09-10-10-31-46-619048/qforecasts.npy")[filter][filter2]
#     # globals
#     res_glm = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/discrete-regression/NBs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # res_gbdt = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/tree-regression/TreeQRsl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     res_fnn = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # globals boosted
#     res_dnnNB = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/transformer__M5__negbin__mean-demand__2024-06-21-13-47-45-072202/forecasts.npy")[filter][filter2]
#     res_dnnTW = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/transformer__M5__tweedie__mean-demand__2024-06-21-13-47-45-074833/forecasts.npy")[filter][filter2]
# elif dataset_name == "OnlineRetail":
#     actuals = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{1}__{dataset_name}/actuals.npy")[filter][filter2]
#     # baselines
#     res_empq = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/EmpQ__OnlineRetail__2024-08-09-09-54-07-013545/qforecasts.npy")[filter][filter2]
#     res_iets = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/iETS__OnlineRetail__2024-09-10-10-25-50-415303/qforecasts.npy")[filter][filter2]
#     # globals
#     res_glm = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/discrete-regression/NBs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # res_gbdt = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/tree-regression/TreeQRsl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     res_fnn = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # globals boosted
#     res_dnnNB = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__OnlineRetail__negbin__mean-demand__2024-06-22-18-04-03-762960/forecasts.npy")[filter][filter2]
#     res_dnnTW = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__OnlineRetail__tweedie__mean-demand__2024-06-22-18-42-17-982152/forecasts.npy")[filter][filter2]
# elif dataset_name == "Auto":
#     actuals = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{1}__{dataset_name}/actuals.npy")[filter][filter2]
#     # baselines
#     res_empq = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/EmpQ__Auto__2024-08-09-09-52-07-174894/qforecasts.npy")[filter][filter2]
#     res_iets = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/iETS__Auto__2024-09-10-10-21-08-184429/qforecasts.npy")[filter][filter2]
#     # globals
#     res_glm = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/discrete-regression/NBs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # res_gbdt = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/tree-regression/TreeQRsl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     res_fnn = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # globals boosted
#     res_dnnNB = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__Auto__negbin__mean-demand__2024-06-22-11-04-18-074667/forecasts.npy")[filter][filter2]
#     res_dnnTW = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__Auto__tweedie__mean-demand__2024-06-22-10-57-08-591613/forecasts.npy")[filter][filter2]
# elif dataset_name == "RAF":
#     actuals = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{1}__{dataset_name}/actuals.npy")[filter][filter2]
#     # baselines
#     res_empq = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/EmpQ__RAF__2024-08-09-09-54-23-309428/qforecasts.npy")[filter][filter2]
#     res_iets = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/iETS__RAF__2024-09-10-10-22-33-114237/qforecasts.npy")[filter][filter2]
#     # globals
#     res_glm = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/discrete-regression/NBs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # res_gbdt = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/tree-regression/TreeQRsl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     res_fnn = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # globals boosted
#     res_dnnNB = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__RAF__negbin__mean-demand__2024-06-23-14-21-47-890960/forecasts.npy")[filter][filter2]
#     res_dnnTW = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__RAF__tweedie__mean-demand__2024-06-23-14-31-11-241740/forecasts.npy")[filter][filter2]
# elif dataset_name == "carparts":
#     actuals = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{1}__{dataset_name}/actuals.npy")[filter][filter2]
#     # baselines
#     res_empq = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/EmpQ__carparts__2024-08-09-09-54-38-830033/qforecasts.npy")[filter][filter2]
#     res_iets = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models_baselines/iETS__carparts__2024-09-10-10-19-39-961801/qforecasts.npy")[filter][filter2]
#     # globals
#     res_glm = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/discrete-regression/NBs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # res_gbdt = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/tree-regression/TreeQRsl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     res_fnn = np.load(f"/Users/nicolo.rubattu/Codebase/iTS/src/dnn-regression/DNNs_newl{lag}__{dataset_name}/qforecasts.npy")[filter][filter2]
#     # globals boosted
#     res_dnnNB = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__carparts__negbin__mean-demand__2024-06-23-10-35-39-750798/forecasts.npy")[filter][filter2]
#     res_dnnTW = np.load(f"/Users/nicolo.rubattu/switchdrive/iTS/trained_models/deepAR__carparts__tweedie__mean-demand__2024-06-23-09-25-09-068940/forecasts.npy")[filter][filter2]

In [5]:
ql_empq = quantile_loss(actuals, res_empq, quantiles=quantiles, avg=False)
ql_iets = quantile_loss(actuals, res_iets, quantiles=quantiles, avg=False)
ql_twgp = quantile_loss(actuals, res_twgp, quantiles=quantiles, avg=False)
ql_glm = quantile_loss(actuals, np.transpose(res_glm, (0,2,1)), quantiles=quantiles, avg=False)
ql_fnn = quantile_loss(actuals, np.transpose(res_fnn, (0,2,1)), quantiles=quantiles, avg=False)
ql_dnnNB = quantile_loss_sample(actuals, res_dnnNB, avg=False)
ql_dnnTW = quantile_loss_sample(actuals, res_dnnTW, avg=False)

for ql in [ql_empq, ql_iets, ql_twgp, ql_glm, ql_fnn, ql_dnnNB, ql_dnnTW]:
    for q in ql.keys():
        ql[q] = fscale(ql[q], q)

In [ ]:
print('glm.nb')
for ql in [ql_glm]:
    for q in ql.keys():
        print("  ",q, np.round(ql[q].mean(),2))
print('fnn.nb')
for ql in [ql_fnn]:
    for q in ql.keys():
        print("  ",q, np.round(ql[q].mean(),2))

In [7]:
stacks = {}
for q in ["QL50", "QL80", "QL90", "QL95", "QL99"]:
    stacks[q] = np.stack([ql[q].mean(axis=1) for ql in [ql_empq, ql_iets, ql_twgp, ql_glm, ql_fnn, ql_dnnNB, ql_dnnTW]]).T
stacks['labels'] = ['EmpQ', 'iETS','TweedieGP', 'GLM.nb', 'FNN.nb', 'DNN.nb', 'DNN.tw']
    
import pickle as pkl
pkl.dump(stacks, open(f'toR/stacks_{dataset_name}.pkl','wb'))

In [8]:
from scipy.stats import rankdata
import matplotlib.pyplot as plt

res = {}
for q in ["QL50", "QL80", "QL90", "QL95", "QL99"]:
    stack = np.stack([ql[q].mean(axis=1) for ql in [ql_empq, ql_iets, ql_twgp, ql_glm, ql_fnn, ql_dnnNB, ql_dnnTW]])
    ranks = np.apply_along_axis(lambda x: rankdata(x, method='min'), axis=0, arr=stack)
    res[q] = (np.mean(ranks, axis=1), np.std(ranks, axis=1))

In [ ]:
fig, axs = plt.subplots(1, 5, figsize=(15,3), sharex=True)

for ax, q in zip(axs, ["QL50", "QL80", "QL90", "QL95", "QL99"]):
    means = res[q][0]
    std_devs = res[q][1]
    x = np.arange(len(means))
    x_labels = ['EmpQ', 'iETS','TweedieGP', 'GLM.nb', 'FNN.nb', 'DNN.nb', 'DNN.tw']
    sorted_indices = np.argsort(means)
    sorted_means = means[sorted_indices]
    sorted_std_devs = std_devs[sorted_indices]
    sorted_labels = [x_labels[i] for i in sorted_indices]
    sorted_lower_std_devs = [s if m-s > 1 else m-1 for m, s in zip(sorted_means, sorted_std_devs)]
    sorted_upper_std_devs = [s if m+s < len(x_labels) else len(x_labels)-m for m, s in zip(sorted_means, sorted_std_devs)]
    y = np.arange(len(sorted_means))
    ax.errorbar(sorted_means, y, xerr=[sorted_lower_std_devs, sorted_upper_std_devs], fmt='D', capsize=3, markersize=4, label="Mean with Error", c="black")
    ax.invert_yaxis()
    ax.set_yticks(y, sorted_labels)
    ax.set_xticks(np.arange(1, len(x_labels)+1))
    ax.set_xlim([0.7,8.3])
    ax.set_title(q)
plt.suptitle(dataset_name)
plt.tight_layout()
plt.savefig(os.path.join("images_ranks",dataset_name+'_ranks.svg'), format='svg')

## error in h

In [ ]:
import pickle as pkl
import matplotlib.pyplot as plt
x = list(range(1,data_info['h']+1))

fig, axs = plt.subplots(1, 4, figsize=(12,3), sharey=True)
for q, ax in zip(["QL80","QL90","QL95","QL99"], axs):
    ax.plot(x, ql_empq[q].mean(axis=0), label='EmpQ', color='#4682B4', linestyle=':')
    ax.plot(x, ql_iets[q].mean(axis=0), label='iETS', color='#1E90FF', linestyle='-')
    ax.plot(x, ql_twgp[q].mean(axis=0), label='TweedieGP', color='magenta', linestyle='-')
    ax.plot(x, ql_glm[q].mean(axis=0), label='glm.nb', color='red')
    ax.plot(x, ql_fnn[q].mean(axis=0), label='fnn.nb', color='orange')
    ax.set_title(r"$\mathbf{sQL_{"+q[2:]+"}}$", fontweight='bold')
    ax.set_xlabel('h')
axs[0].legend(ncols=2)
plt.tight_layout()
plt.savefig(os.path.join("images",dataset_name+'_herror.svg'), format='svg')
ylim = axs[0].get_ylim()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12,3), sharey=True)
for q, ax in zip(["QL80","QL90","QL95","QL99"], axs):
    ax.plot(x, ql_twgp[q].mean(axis=0), label='TweedieGP', color='magenta', linestyle='-')
    # ax.plot(x, ql_fnn[q].mean(axis=0), label='fnn.nb', color='orange')
    ax.plot(x, ql_dnnNB[q].mean(axis=0), label='dnn.nb', color='green')
    ax.plot(x, ql_dnnTW[q].mean(axis=0), label='dnn.tw', color='lightgreen')
    ax.set_title(r"$\mathbf{sQL_{"+q[2:]+"}}$", fontweight='bold')
    ax.set_xlabel('h')
ax.set_ylim((ax.get_ylim()[0], ylim[1]))
axs[0].legend(ncols=2)
plt.tight_layout()
plt.savefig(os.path.join("images",dataset_name+'_herrordnn.svg'), format='svg')

# coverage

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(4,4))

axs.plot(quantiles, quantiles, color="black", linestyle='--', linewidth=0.8)

axs.plot(quantiles, [(actuals <= res_empq[:,:,i]).mean() for i in list(range(5))], label="EmpQ", marker='o', markersize=2, color='#4682B4', linestyle=':')
axs.plot(quantiles, [(actuals <= res_iets[:,:,i]).mean() for i in list(range(5))], label="iETS", marker='o', markersize=2, color='#1E90FF')
axs.plot(quantiles, [(actuals <= res_twgp[:,:,i]).mean() for i in list(range(5))], label="TweedieGP", marker='o', markersize=2, color='magenta')
axs.plot(quantiles, [(actuals <= np.transpose(res_glm, (0,2,1))[:,:,i]).mean() for i in list(range(5))], label="glm.nb", marker='o', markersize=2, color='red')
axs.plot(quantiles, [(actuals <= np.transpose(res_fnn, (0,2,1))[:,:,i]).mean() for i in list(range(5))], label="fnn.nb", marker='o', markersize=2, color='orange')
axs.plot(quantiles, [(actuals <= np.quantile(res_dnnNB, quantiles, axis=1, method="higher")[i,:,:]).mean() for i in list(range(5))], label="dnn.nb", markersize=2, color='green')
axs.plot(quantiles, [(actuals <= np.quantile(res_dnnTW, quantiles, axis=1, method="higher")[i,:,:]).mean() for i in list(range(5))], label="dnn.tw", markersize=2, color='lightgreen')

axs.set_xlabel('target level')
axs.set_ylabel('forecast coverage')
axs.set_title(f"{dataset_name}", fontweight="bold")

axs.grid(color='lightgray', linewidth=0.5)
axs.legend()
plt.tight_layout()
plt.savefig(os.path.join("coverage",dataset_name+'_cov.svg'), format='svg')

# Forecast plot

In [ ]:
import pandas as pd
from tqdm import tqdm

for i in tqdm(range(len(datasets['test']))):
    ts = np.array(datasets['test'][i]['target'])
    dt = pd.period_range(start=datasets['test'][i]['start'], periods=ts.size)
    ts = pd.Series(index=dt, data=ts)
    dt_test = ts.index[-data_info['h']:]
    ts = ts[-data_info['h']*3:]
    fig, axs = plt.subplots(7,1, figsize=(8,8), sharex=True, sharey=True)
    for method, ax, res in zip(
        ["EmpQ", "iETS", "TweedieGP", "glm.nb","fnn.nb", "dnn.nb", "dnn.tw"],
        axs,
        [res_empq, res_iets, res_twgp, np.transpose(res_glm, (0,2,1)), np.transpose(res_fnn, (0,2,1)), res_dnnNB, res_dnnTW]
    ):
        if method[:3] == 'dnn': res = np.transpose(np.quantile(res, quantiles, axis=1, method="higher"), (1,2,0))
        ts.plot(ax=ax, color="black")
        old_y = -1
        for qi, q in enumerate(list(reversed(["QL50", "QL80", "QL90", "QL95", "QL99"]))):
            qi = 4 - qi
            ax.fill_between(dt_test, np.ceil(res[i,:,qi]), [0]*data_info['h'], color="blue", alpha=0.2)
            if (np.ceil(res[i,-1,qi]) != old_y):
                ax.text(dt_test[-1], np.ceil(res[i,-1,qi]), r"$\,\,q="+str(int(q[2:])/100)+"$", ha="left", va="center")
            old_y = np.ceil(res[i,-1,qi])
    for method, ax in zip(["EmpQ", "iETS", "TweedieGP", "glm.nb","fnn.nb", "dnn.nb", "dnn.tw"], axs):
        ax.text(ts.index[0], ax.get_ylim()[1]*0.95, " " + method, ha="left", fontweight="bold", va="top")
    fig.suptitle(i)
        
    plt.tight_layout()
    plt.savefig(os.path.join("forecastplots",dataset_name+f'_{i}.jpeg'), format='jpeg')
    plt.close()